# 🧑‍💻 Session 6: Retrieval Augmented Generation (RAG)

RAG combines:
1. **Retriever** → Fetches relevant documents from a vector store.  
2. **LLM** → Generates context-aware responses using query + retrieved docs.  

In this session, we will:
- Store documents with **Gemini Embeddings**  
- Retrieve docs from **Chroma**  
- Use **Groq LLM** for answering questions  


In [1]:
# 📌 Install dependencies
!pip install -qU langchain langchain-community langchain-groq langchain-google-genai chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.7 MB/s eta 0:00:00

## 🔑 Setup API Keys
- Google Gemini API key → for embeddings  
- Groq API key → for LLM (LLaMA models)


In [ ]:
import os
os.environ["NO_GCE"] = "true"  # Add this to disable metadata service checks in Colab
os.environ["GOOGLE_API_KEY"] = "[YOUR_GEMINI_API_KEY]"  # Replace with your actual Google API key

## 📄 Step 1: Create Documents
We’ll use IPL players knowledge base.


In [3]:
from langchain.schema import Document

docs = [
    Document(
        page_content="Virat Kohli is one of the most successful batsmen in IPL history and has captained RCB.",
        metadata={"team": "Royal Challengers Bangalore"}
    ),
    Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, winning five titles with Mumbai Indians.",
        metadata={"team": "Mumbai Indians"}
    ),
    Document(
        page_content="MS Dhoni has led Chennai Super Kings to multiple IPL titles and is known as Captain Cool.",
        metadata={"team": "Chennai Super Kings"}
    ),
    Document(
        page_content="Jasprit Bumrah is a leading fast bowler for Mumbai Indians, famous for his yorkers.",
        metadata={"team": "Mumbai Indians"}
    ),
    Document(
        page_content="Ravindra Jadeja is an all-rounder for Chennai Super Kings, contributing with bat, ball, and fielding.",
        metadata={"team": "Chennai Super Kings"}
    )
]


## 🗂️ Step 2: Store Documents with Gemini Embeddings in Chroma
We’ll use **GoogleGenerativeAIEmbeddings** for vector representation.


In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Create Chroma vector store
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="rag_chroma_db",
    collection_name="ipl_docs"
)

# Add documents
vector_store.add_documents(docs)


['0a1713af-ce1f-4bcf-96fe-3e4f0da8d891',
 'bd643bd2-78e6-4e1e-b318-c32747104443',
 'f9a9932b-61d1-4f99-9155-53d616189d0e',
 'e0676031-f087-4b2c-a8dc-8c5ea23409fb',
 'b545d6c3-6e49-43a5-97a9-7a0ec92d2c68']

## 🔎 Step 3: Create Retriever
Retriever fetches relevant chunks from Chroma.


In [6]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})


## 🧠 Step 4: Initialize Groq LLM
We use Groq-hosted LLaMA 3.


In [8]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_groq import ChatGroq

# Load API key
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
llm = ChatGroq(
    model="openai/gpt-oss-20b",
    api_key=GROQ_API_KEY,
    temperature=0.3,
    max_tokens=200
)



## 🔗 Step 5: Create RAG Chain
Combine retriever + LLM into a RetrievalQA pipeline.


In [9]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


## 💬 Step 6: Ask Questions
Test RAG pipeline with cricket-related queries.


In [10]:
# Query 1
query = "Who is the most successful IPL captain?"
response = qa_chain.invoke({"query": query})

print("Query:", query)
print("Answer:", response["result"])
print("\nSources:", [doc.metadata for doc in response["source_documents"]])

# Query 2
query2 = "Which bowler is famous for yorkers?"
response2 = qa_chain.invoke({"query": query2})

print("\nQuery:", query2)
print("Answer:", response2["result"])
print("\nSources:", [doc.metadata for doc in response2["source_documents"]])


Query: Who is the most successful IPL captain?
Answer: The most successful IPL captain is **Rohit Sharma**, who has led Mumbai Indians to five titles.

Sources: [{'team': 'Mumbai Indians'}, {'team': 'Royal Challengers Bangalore'}]

Query: Which bowler is famous for yorkers?
Answer: The bowler famous for his yorkers is **Jasprit Bumrah**.

Sources: [{'team': 'Mumbai Indians'}, {'team': 'Mumbai Indians'}]


# ✅ Summary
- Used **Google Gemini Embeddings** to vectorize documents  
- Stored + Retrieved docs from **Chroma**  
- Connected retriever with **Groq LLM**  
- Answered queries using **RAG pipeline**  

👉 Next session: **Advanced RAG – Custom Prompts, Comparisons, and Deep Dive**
